In [15]:
import mysql.connector
import pandas as pd

Connecting to database

In [5]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="my-secret-pw",
  database="droptime"
)

In [6]:
mycursor = mydb.cursor()

quering

In [24]:
query = """
SELECT *, TIMESTAMPDIFF(SECOND, segment_start_time, segment_end_time) AS time_diff_in_seconds 
FROM route_segments
ORDER BY driver_id, segment_id DESC, segment_end_time ASC
"""
df = pd.read_sql(query, mydb)

/tmp/ipykernel_122979/81233910.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, mydb)


In [25]:
df.head()

,segment_id,driver_id,segment_type,order_id,segment_start_time,segment_end_time,time_diff_in_seconds
0,4973,1,DRIVE,NaN,2024-02-18 18:22:33,2024-02-18 18:28:46,373
1,4972,1,STOP,1950.0,2024-02-18 18:31:46,2024-02-18 18:32:46,60
2,4949,1,STOP,NaN,2024-02-01 05:07:48,2024-02-01 05:19:08,680
3,4948,1,DRIVE,NaN,2024-02-01 05:15:22,2024-02-01 05:24:53,571
4,4947,1,STOP,2080.0,2024-02-01 05:27:53,2024-02-01 05:29:03,70
